In [19]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [20]:
imgPaths = glob('./Selected/*.jpeg')

In [26]:
#extract information from one card
imgPath = imgPaths[0]
print(imgPath)
_, filename = os.path.split(imgPath)
print(filename)
# extract data and text
#read image on opencv
image = cv2.imread(imgPath)
data = pytesseract.image_to_data(image)
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])
#drop missing value
df.dropna(inplace=True)
df['conf'] = df['conf'].astype(float).astype(int)
useFulData = df.query('conf >= 30')
# Dataframe
businessCard = pd.DataFrame()
businessCard['text'] = useFulData['text']
businessCard['id'] = filename

./Selected\000.jpeg
000.jpeg


In [27]:
businessCard

,text,id
4,,000.jpeg
9,040-4852,000.jpeg
10,"8881,",000.jpeg
11,90309,000.jpeg
12,52549,000.jpeg
14,Fi,000.jpeg
15,/laurelsoverseaseducation,000.jpeg
20,LAURELS,000.jpeg
24,OVERSEAS,000.jpeg
25,EDUCATIONAL,000.jpeg


In [29]:
#extract information from all card
allBusinessCard = pd.DataFrame(columns=['id', 'text'])

for imgPath in tqdm(imgPaths, desc='BusinessCard'):
    _, filename = os.path.split(imgPath)
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'), data.split('\n')))
    
    df = pd.DataFrame(dataList[1:], columns=dataList[0])
    df.dropna(inplace=True)

    try:
        df['conf'] = df['conf'].astype(float)
        useFulData = df.query('conf >= 30')
        
        businessCard = pd.DataFrame()
        businessCard['text'] = useFulData['text']
        businessCard['id'] = filename
        
        allBusinessCard = pd.concat((allBusinessCard, businessCard), ignore_index=True)
    except Exception as e:
        print(f"Erreur pour l'image {filename} : {e}")


BusinessCard: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 293/293 [02:39<00:00,  1.84it/s]


In [31]:
#save data extracted on csv file
allBusinessCard.to_csv('businessCard.csv',index=False)